# Running Biomechanics Data Set

Reginaldo K Fukuchi, Jan 2023, reginaldo.fukuchi@ufabc.edu.br

This NB imports c3d files and convert data into ASCII files for markers trajectories and forces separately.

In [1]:
# Prepare environment
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

### Helper functions

In [2]:
def difference (list1, list2):
    """Compare two lists for differences"""
    list_dif = [i for i in list1 + list2 if i not in list1 or i not in list2]
    
    return list_dif

#### Function to create a dataframe with the same structure as the text files
##### Markers

In [89]:
def c3d2txt_mkrs(c3d_out, label_ls):
    """Convert c3d data to ASCII data keeping the same structure as the text files in
    the RBDS Figshare repository
    """
    # Rename dictionary
    c = c3d_out
    # Assign variables
    point_data       =c['data']['points']# markers data
    points_residuals =c['data']['meta_points']['residuals']
    analog_data      =c['data']['analogs']# analog data
    mkr_S_labels_RBDS=c['parameters']['POINT']['LABELS']['value']# marker labels

    ## Check for differences in marker columns btw RBDS Figshare and new data
    list_dif = difference (label_ls, mkr_S_labels_RBDS)
    # Delete columns of markers that are not currently in the RBDS Figshare
    if list_dif:
        idx = [mkr_S_labels_RBDS.index(i) for i in list_dif if i in mkr_S_labels_RBDS]
        point_data = np.delete(point_data, idx, axis=1) # Delete data corresponding to the non-existent marker
        mkr_S_labels_RBDS = [ele for ele in mkr_S_labels_RBDS if ele not in list_dif]

    # Delete extra dimension for homogeneus matrices
    point_data=np.delete(point_data, 3, axis=0)

    pos_data = np.empty(shape=(point_data.shape[2],3*len(mkr_S_labels_RBDS)))
    for m, marker in enumerate(mkr_S_labels_RBDS):
        pos_data[:,3*m:3*m+3] = point_data[:, m, :].T

    # Create new list to be column names of df
    xyz = list('XYZ')*len(mkr_S_labels_RBDS)
    mkr_S_labels_RBDS_2 = [ele for ele in mkr_S_labels_RBDS for i in range(3)]
    mkr_S_labels_RBDS_3 = [mkr_S_labels_RBDS_2[i]+xyz[i] for i in range(len(xyz))]

    df_S_RBDS = pd.DataFrame(data=pos_data, columns=mkr_S_labels_RBDS_3,)

    duration  = pos_data.shape[0]/int(c['parameters']['POINT']['RATE']['value'][0])
    timeStamps= np.linspace(0, duration, pos_data.shape[0]).round(3)

    df_S_RBDS['Time'] = timeStamps
    df_S_RBDS.set_index('Time', inplace=True)
    df_S_RBDS.index.name = 'Time'
    # Precision=3
    df_S_RBDS = df_S_RBDS.round(3)
    
    return df_S_RBDS

##### Forces

In [4]:
def c3d2txt_grfs(c3d_out):
    """Convert c3d data to ASCII data keeping the same structure as the text files in
    the RBDS Figshare repository
    """
    # Rename dictionary
    c = c3d_out
    
    # Select the second platform
    pf_1 = c["data"]["platform"][1]
    # Force platform signals matching existing RBDS
    fp1_forces = pf_1['force'].T # Forces
    fp1_cop    = pf_1['center_of_pressure'].T # Centre of pressure
    # Free moment about vertical axis
    fp1_Ty     = pf_1['Tz'][1,:] 
    fp1_Ty = np.expand_dims(fp1_Ty,axis=1)

    # Concatenate signals
    force_signals = np.hstack([fp1_forces, fp1_cop, fp1_Ty])

    force_sigs_labels = ['Fx', 'Fy', 'Fz', 'COPx', 'COPy', 'COPz', 'Ty']

    df_forces = pd.DataFrame(data=force_signals, columns=force_sigs_labels)

    df_forces['Time'] = list(range(1,force_signals.shape[0]+1))
    df_forces.set_index('Time', inplace=True)
    df_forces.index.name = 'Time'

    df_forces = df_forces.round(2)
    
    return df_forces

## Import C3D RBDS file using EZC3D
https://github.com/pyomeca/ezc3d

In [5]:
from ezc3d import c3d

In [6]:
# Import data
dir_O = r'C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\RBA' # Directory of origin
dir_D = r'C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare_update' # Directory of destination

### List of markers labels that are used in the RBDS Figshare data set

In [7]:
mkr_labels_S= ['R.ASIS', 'L.ASIS', 'R.PSIS', 'L.PSIS', 'R.Iliac.Crest', 'L.Iliac.Crest', 
                'R.Thigh.Top.Lateral', 'R.Thigh.Bottom.Lateral', 'R.Thigh.Top.Medial', 'R.Thigh.Bottom.Medial', 
                'R.Shank.Top.Lateral', 'R.Shank.Bottom.Lateral', 'R.Shank.Top.Medial', 'R.Shank.Bottom.Medial', 
                'R.Heel.Top', 'R.Heel.Bottom', 'R.Heel.Lateral', 'L.Thigh.Top.Lateral', 'L.Thigh.Bottom.Lateral', 
                'L.Thigh.Top.Medial', 'L.Thigh.Bottom.Medial', 'L.Shank.Top.Lateral', 'L.Shank.Bottom.Lateral', 
                'L.Shank.Top.Medial', 'L.Shank.Bottom.Medial', 'L.Heel.Top', 'L.Heel.Bottom', 'L.Heel.Lateral', 
                'R.GTR', 'R.Knee', 'R.Knee.Medial', 'R.HF', 'R.TT', 'R.Ankle', 'R.Ankle.Medial', 'R.MT1', 'R.MT5', 
                'R.MT2', 'L.GTR', 'L.Knee', 'L.Knee.Medial', 'L.HF', 'L.TT', 'L.Ankle', 'L.Ankle.Medial', 
                'L.MT1', 'L.MT5', 'L.MT2']

mkr_labels_R=['R.ASIS', 'L.ASIS', 'R.PSIS', 'L.PSIS', 'R.Iliac.Crest', 'L.Iliac.Crest', 
              'R.Thigh.Top.Lateral', 'R.Thigh.Bottom.Lateral', 'R.Thigh.Top.Medial', 'R.Thigh.Bottom.Medial', 
              'R.Shank.Top.Lateral', 'R.Shank.Bottom.Lateral', 'R.Shank.Top.Medial', 'R.Shank.Bottom.Medial', 
              'R.Heel.Top', 'R.Heel.Bottom', 'R.Heel.Lateral', 
              'L.Thigh.Top.Lateral', 'L.Thigh.Bottom.Lateral', 'L.Thigh.Top.Medial', 'L.Thigh.Bottom.Medial', 
              'L.Shank.Top.Lateral', 'L.Shank.Bottom.Lateral', 'L.Shank.Top.Medial', 'L.Shank.Bottom.Medial', 
              'L.Heel.Top', 'L.Heel.Bottom', 'L.Heel.Lateral', 
              'R.MT1', 'R.MT5', 'L.MT1', 'L.MT5']

In [106]:
subject='SUB0052'

In [107]:
fname_S = glob.glob(os.path.join(dir_O,subject,'static.c3d'))# static file pathname
fname_S

['C:\\Users\\Reginaldo\\Documents\\data\\CNPq\\RBDS_v2\\RBA\\SUB0052\\static.c3d']

In [108]:
c_S = c3d(fname_S[0])
df_S_RBDS = c3d2txt_mkrs(c_S, mkr_labels_S)
df_S_RBDS.head()

,R.ASISX,R.ASISY,R.ASISZ,L.ASISX,L.ASISY,L.ASISZ,R.PSISX,R.PSISY,R.PSISZ,L.PSISX,...,L.Ankle.MedialZ,L.MT1X,L.MT1Y,L.MT1Z,L.MT5X,L.MT5Y,L.MT5Z,L.MT2X,L.MT2Y,L.MT2Z
Time,,,,,,,,,,,,,,,,,,,,,
0.000,961.613,1015.880,1360.445,961.069,1011.396,1122.474,779.256,1054.549,1282.965,780.059,...,1132.508,936.807,59.455,1134.652,917.708,51.014,1025.094,946.217,69.691,1093.821
0.007,961.608,1015.878,1360.460,961.073,1011.387,1122.482,779.256,1054.534,1282.987,780.073,...,1132.506,936.811,59.459,1134.644,917.697,51.004,1025.093,946.220,69.703,1093.826
0.013,961.605,1015.877,1360.482,961.076,1011.380,1122.504,779.252,1054.547,1282.993,780.083,...,1132.504,936.809,59.450,1134.641,917.690,51.005,1025.094,946.222,69.698,1093.815
0.020,961.607,1015.881,1360.482,961.069,1011.378,1122.517,779.264,1054.532,1282.995,780.087,...,1132.512,936.823,59.463,1134.638,917.702,50.999,1025.092,946.218,69.694,1093.821
0.027,961.597,1015.888,1360.515,961.071,1011.368,1122.539,779.251,1054.540,1283.005,780.098,...,1132.515,936.803,59.452,1134.649,917.690,51.012,1025.079,946.222,69.707,1093.830


In [109]:
# Export to csv
fname_out = os.path.join(dir_D, 'RBDS0'+subject[-2:]+'static.txt')
df_S_RBDS.to_csv(fname_out, sep='\t')

In [110]:
fname_R = glob.glob(os.path.join(dir_O,subject,'runT*.c3d'))# dynamic files pathname
fname_R

['C:\\Users\\Reginaldo\\Documents\\data\\CNPq\\RBDS_v2\\RBA\\SUB0052\\runT351.c3d',
 'C:\\Users\\Reginaldo\\Documents\\data\\CNPq\\RBDS_v2\\RBA\\SUB0052\\runT352.c3d']

In [105]:
for f, fname in enumerate(fname_R):
    c_R = c3d(fname_R[0], extract_forceplat_data=True)
    
    # Create a df with markers data
    df_R_RBDS = c3d2txt_mkrs(c_R, mkr_labels_R)
    # Create a df with forces data
    df_forces = c3d2txt_grfs(c_R)
    # Export to CSV
    fname_out_R = os.path.join(dir_D, 'RBDS0'+subject[-2:]+fname[59:65]+'markers.txt')
    df_R_RBDS.to_csv(fname_out_R, sep='\t')
    # Export to CSV
    fname_forces_txt = os.path.join(dir_D, 'RBDS0'+subject[-2:]+fname[59:65]+'forces.txt')
    df_forces.to_csv(fname_forces_txt, sep='\t')